In [18]:
#testget_gnd_state_probs_and_approx_ratio.ipynb
from pyaqc.RCModules.aqc_rigetti import get_gnd_state_probs_and_approx_ratio
#ONE AMBULANCE adjacency table
from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateAmbulanceAdjacency,distance #   qc_all_env and rigoct38_env
import numpy as np 
import unicodedata as ud
betagreek =  ud.lookup('GREEK SMALL LETTER BETA')
gammagreek =  ud.lookup('GREEK SMALL LETTER gamma')

################  Set up type of problem to solve ################
n_destinations = 5
n_qubits = n_destinations
HammingWeightOfConstraint =  n_qubits - 1

gridWidth = n_qubits
ConstraintMultiplier = 14             # I have used 14 for 5 qubit with constraints and traditional mixer, 140 for q11, tim 200
# 1500 for q17
remove_constraint=  1   # remove the constraints when using an XY mixer
select_qubo_model = 1   # we nearly always use this
################ Create 1) 'Adjacency' table of the problem, and 2) 'Adjacency_constraint' of the problems constraints 3) Adjacency_feasible ################
#'Adjacency' table of the whole problem

Adjacency = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 1,
    remove_constraint= remove_constraint, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
#Adjacency of just the problems constraints
Adjacency_constraint = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 0,
    remove_constraint= remove_constraint, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
# Adjacency that defines whether a given state is 'feasible'
Adjacency_feasible = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 0,
    remove_constraint= 0, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
################ ... and print ################
if select_qubo_model:print('Qubo ', end='')
else: print('Ising ', end='')
print_QUBOdetails(Adjacency,n_qubits, 'Ambulance problem')
n_ambulance=1
Ansatz_type = {'n_qubits':n_qubits, 'p':0,'Mixer':0,'prog_init':0, 'ConstraintMultiplier':0, 'remove_constraint':0, 'Optimizer_maxiter': 0}
Ansatz_type['ConstraintMultiplier'] = ConstraintMultiplier
Ansatz_type['remove_constraint'] = remove_constraint

Qubo ADJACENCY MATRIX Ambulance problem  

Qubit  q0   q1   q2   q3   q4 

  q0   0    -1   -4   -9  -16 

  q1             -1   -4   -9 

  q2                  -1   -4 

  q3                       -1 



In [16]:
#TWO AMBULANCE DISTANCE MINIMISATION - 


#################################### Create Adjacency for a qubo problem of two ambulances ####################################
import numpy as np 
import unicodedata as ud
betagreek =  ud.lookup('GREEK SMALL LETTER BETA')
gammagreek =  ud.lookup('GREEK SMALL LETTER gamma')
#RC utility functions    # C:\Users\Robert\Anaconda3\envs\rigoct38_env\Lib\site-packages\pyaqc\RCModules\
from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateTwoAmbulanceAdjacencyV2

n_destinations = 4  
gridWidth = n_destinations
HammingWeightOfConstraint = n_destinations
remove_constraint=  0   # remove most of the hard constraints when using an XY mixer
ConstraintMultiplier= 10
result = CreateTwoAmbulanceAdjacencyV2(gridWidth,n_destinations, Adddistance = 1,ConstraintMultiplier=ConstraintMultiplier, use_XYMixer_constraints = remove_constraint)
Adjacency = result['qubo']
quboHybrid = result['quboHybrid']
n_qubits = result['n_qubits']
ConstraintMultiplier = result['ConstraintMultiplier']
max_distance = result['max_distance']
sum_distance = result['sum_distance']
# Adjacency that defines whether a given state is 'feasible'.This is needed to calculate the 'approximation ratio' arXiv:1904.09314v2 [quant-ph] 21 May 2020 
result_cst = CreateTwoAmbulanceAdjacencyV2(gridWidth,n_destinations, Adddistance = 0,ConstraintMultiplier=ConstraintMultiplier, use_XYMixer_constraints = remove_constraint)
Adjacency_feasible = result_cst['qubo']
#'Adjacency' table of the whole problem
#Adjacency of just the problems constraints


select_qubo_model = 1  # This is automatically the case in the function CreateTwoAmbulanceAdjacencyV2
Width  = gridWidth
Height = n_destinations//gridWidth
n_ambulance=2
Ansatz_type = {'n_qubits':n_qubits, 'p':0,'Mixer':0,'prog_init':0, 'ConstraintMultiplier':ConstraintMultiplier, 'remove_constraint':0, 'Optimizer_maxiter': 0}
if 1:
    filename = 'Twoambulances.txt'
    print('Qubo ', end='')
    print_QUBOdetails(result_cst['qubo'],n_qubits,filename)
    print(Ansatz_type)

Qubo ADJACENCY MATRIX Twoambulances.txt  

Qubit  q0   q1   q2   q3   q4   q5   q6   q7   q8   q9  q10  q11  q12  q13  q14  q15 

  q0  -10                  20 

  q1       -10                  20 

  q2            -10                  20 

  q3                 -10                  20 

  q4                      -10 

  q5                           -10 

  q6                                -10 

  q7                                     -10 

  q8                                          -10   20   20   20 

  q9                                               -10   20   20 

 q10                                                    -10   20 

 q11                                                         -10 

 q12                                                              -10   20   20   20 

 q13                                                                   -10   20   20 

 q14                                                                        -10   20 

 q15                     

In [31]:
 ################# Define functions to create 1) initial state, 2) Mixer circuit 3) Phase separating circuit ############################
# Get the Quantum circuit of the initial state (eg Hamamard for Xmixer)
#Get Mixer quantum circuit
#Get Pauli observables that represent an Ising problem, derived from an Adjacency table using a qubo definition of edge.

from pyaqc.RCModules.aqc_rigetti import create_ring_mixer, Adjacency_qubo_to_Regetti,Adjacency_Ising_to_Regetti,create_local_ring_mixer,Dicke_state, Dicke_state_local,get_gnd_state_probs_and_approx_ratio,ansatz_init_XYMixer,ansatz_prog_init,ansatz_EV_init
from pyquil.paulis import *  #  exponential_map, exponentiate_commuting_pauli_sum
from pyquil.api import *
from pyquil.gates import *
"""
def get_mixer(n_qubits,use_XY_mixer,n_ambulance )
def get_prog_init(n_qubits,use_XY_mixer,n_ambulance )

def get_approx_ratio_init(ansatz_prog,prog_init,ListPauli_termsMy,SumPauli_termsMy,n_qubits,p,Adjacency_constraint,Adjacency_feasible,  prt_details=1,state_feasible=None )

def get_gnd_state_probs_and_approx_ratio(opt_betagamma,  prt_details=1)
"""
def get_mixer(n_qubits,use_XY_mixer,n_ambulance, print_details=True ):
    """
    Return Rigetti observables that can be exponentiated, and a description 'Ansatz_type' (type:dict)
    type: Pauli Observables, for example sX(n) can be exponentiated e^{-beta sum_0^n(sX(n))} which causes a X axis rotation of beta for all n qubits.
    if use_XY_mixer==1 then the mixer created is an XY ring mixer otherwise it is an X mixer 
    if n_ambulance ==2 and use_XY_mixer==1, then 3 XY ring mixers are created otherwise it is an X mixer
    if,
        mixer_{j,k} = sX(j)sX(k) + sY(j)sY(k)
    we can define a ring mixer length n as 
        ring_mixer = mixer_{0,n} + sum_{j=0}^{n-1}(mixer_{j,j+1)
    and create the quantum circuit for an XY ring mixer
        e^{-gamma ring_mixer}
    """
    MyMixerHam = []
    MyMixerHam.append(0)                # Perhaps required by QAOA()
    if not use_XY_mixer:           # when the extra hard constraints are in place use an X mixer, otherwise an XY mixer
        for n in range(n_qubits):       
            MyMixerHam[0] += 1*sX(n) 
        Ansatz_type['Mixer'] ='TRADITIONAL /  Farhi  mixer'
    else:
        XYmixer = create_ring_mixer(n_qubits)
        MyMixerHam[0] = XYmixer         #XY
        Ansatz_type['Mixer'] ='XY Ring mixer'
        if n_ambulance==2:
            MyMixerHam, prog_init_three_XYMixers = three_XYMixers_Dicke_states(n_qubits, n_destinations) 
            Ansatz_type['prog_init'] = 'Three Dicke states'
            Ansatz_type['Mixer'] ='Three XY Ring mixers' 
    
    if n_ambulance==2:
        labels= ['Destination XYmixer','Start0 XYmixer','Start1 XYmixer']
        for n_mixer, mixer in enumerate(MyMixerHam):
            if print_details: 
                print(labels[n_mixer])
                print(mixer)
    else: 
        if print_details: 
            print(Ansatz_type['Mixer'])
            print(MyMixerHam[0])        
        
    return MyMixerHam, Ansatz_type
def get_prog_init(n_qubits,use_XY_mixer,n_ambulance ):
    """
    Return the Quantum circuit of the initial state (eg Hamamard for Xmixer)
    if use_XY_mixer==1 then the circuit created is a Dicke_state otherwise it is an Hamamard
    if n_ambulance ==2 then and use_XY_mixer==1, then 3  Dicke_states are created otherwise it is       a Hamamard.
    """
    prog_init = Program()

    if not use_XY_mixer:
        #A) 
        print('Hadamard initial state.   ', n_qubits, ' = n_qubits')
        for n in range(n_qubits):
            prog_init += H(n)
    else:
        if n_ambulance==2:
            #B)
            #MyMixerHam, prog_init_three_XYMixers = three_XYMixers_Dicke_states(n_qubits, n_destinations)  
            prog_init = prog_init_three_XYMixers       # this is just a reminder that the calculation was made earlier in the 'Mixer used' cell
            print('The initial state is made of three separate Dicke_state s')
        else:
            #C)
            print('Dicke_state initial state',HammingWeightOfConstraint, ' = HW')
            prog_init = Dicke_state(n_qubits, HammingWeightOfConstraint)

    return prog_init

from pyaqc.RCModules.QAOARC import decimal_state_to_binary_reversed, Energy_of_binary_state
from pyaqc.RCModules.aqc_rigetti import min_energy
def get_approx_ratio_init(ansatz_prog,prog_init,ListPauli_termsMy,SumPauli_termsMy,n_qubits,p,Adjacency_constraint,Adjacency_feasible,  prt_details=1,state_feasible=None ):
    """
    The energy of , Adjacency_feasible (the X-mixer constraint adjacency), in a state_feasible, will be the characteristic energy that all feasible states share. 
    
    The energy of Adjacency_constraint in its ground state, is the energy cost of complying with all constraints in the problem
    
    Initialise the function, 'get_gnd_state_probs_and_approx_ratio(opt_betagamma)' , in one of two ways
        1) Each call will be for the same ising problem. Hence, it will use the same E_min_feas,  E_max_feas, and min_energy_constraint
        2) each call of 'get_gnd_state_probs_and_approx_ratio' can use a different betagamma,  otherwise a p based on opt_betagamma
    """
    
    if isinstance(state_feasible,str):
        state_feasible=state_feasible.replace(' ','')
    "get  Minimum energy of the Adjacency by considering all 2^n_qubits states. Or more quickly using a suppied state_feasible"
    energy_of_feasible_state = min_energy( Adjacency_feasible, n_qubits, state_feasible=state_feasible)      ##use An adjacency that defines feasible space
    min_energy_constraint_problem = min_energy( Adjacency_constraint, n_qubits, state_feasible=state_feasible)              ##use problem's actual mixer_constraints
    
    ### Calculate E_min_feas and E_max_fease  by a 2^n search ##############################################
    ListPauli_terms_Xconstr,SumPauli_terms_Xconstr  = Adjacency_qubo_to_Regetti( Adjacency_feasible, n_qubits)
    def energy_min_max_feasible():
        seen_first_compliant_state = 0
        E_min_feas = Energy_of_binary_state( 0, SumPauli_termsMy,n_qubits)-min_energy_constraint_problem
        for n in range(2**n_qubits):
            E_feas_of_state = Energy_of_binary_state( n, SumPauli_termsMy,n_qubits)-min_energy_constraint_problem
            if Energy_of_binary_state( n, SumPauli_terms_Xconstr, n_qubits) == energy_of_feasible_state:
                if seen_first_compliant_state==0:
                    E_max_feas = E_feas_of_state
                E_max_feas = max(E_max_feas,E_feas_of_state)
                E_min_feas = min(E_min_feas,E_feas_of_state)
                seen_first_compliant_state = 1
        return E_min_feas, E_max_feas 

    E_min_feas, E_max_feas = energy_min_max_feasible()
    ##############################################
    if prt_details==1:
        print('E_min_feas = ', E_min_feas,'\nE_max_feas = ', E_max_feas,'\nenergy_of_feasible_state = ', energy_of_feasible_state,'\nmin_energy_constraint_problem = ',min_energy_constraint_problem  )
    
    def get_gnd_state_probs_and_approx_ratio(opt_betagamma,  prt_details=1):  
        """prob_gnd_state, approx_ratio, prob_feasible_state
         
         return A)'prob_gnd_state' the cumulative probability of the lowest energy state(s) sampled, 
                
                B) 'prob_feasible_state' The probability that one of the suggested states is a feasible state (defined as states that minimize 'Adjacency_feasible') 
         
                C) approx_ratio = (EV_feasible/prob_feasible_state-E_max_feas) /(E_min_feas-E_max_feas)
                    Where 
                    E_min_feas = the problem's minimum energy in the feasible space with all constraint energies set to 0.
                    E_max_feas = the problem's maximum energy in the feasible space with all constraint energies set to 0.
                    EV_feasible = the expected energy of suggested states that are feasible
                        As defined by Wang in arXiv:1904.09314v2 [quant-ph] 21 May 2020, page 3 col 2
        type: float,float,float

        param
            ansatz_prog a quantum circuit eg for p=1, e^{beta * H_mixer} e^{gamma * H_cost} prog_init
                type:function that returns a Rigetti.Program()
            'Adjacency_constraint'  The adjacency table which contains only the problem's constraint energies , and no Soft energies (eg distance)
            This allows the constraint energies to be calculated separately from the non-constraint energies.
                type: dict example {(0, 0): 5,  (1, 1): -1,  (2, 2): -1}
            
        kwargs
            
            'state_feasible' a string of a state that complies with all the constraints in Adjacency_constraint. For example;  '11011'. When supplied the minimum energy of                 all the constraints is calculated in one calculation, rather than 2^n_qubits calculations. This speeds up the approximation ratio calculation
                type: string of a binary state
            
            
        if p == len(opt_betagamma)//2=False, the ansatz_prog is re initiated each call of get_gnd_state_probs_and_approx_ratio, to reflect the p implied by opt_betagamma (eg when creating the approx ratio for many different p s of the same problem)
        if p == len(opt_betagamma)//2=True , execution time is saved by using the ansatz_prog that was supplied by get_approx_ratio_init (eg when looking for betagamma minimum)
        In the feasible space, what is the average energy found after substituting the actual constraint energy with 0? 
        """
        prob_gnd_state = 0
        prob_feasible_state = 0
        EV_feasible = 0 
        EV_not_feasible = 0 
        n_feasible = 0
        approx_ratio = -1
        EV_all_suggestions = 0
        Expectation_of_state = 0
        
        if p == len(opt_betagamma)//2:
            # calculate solution_ansatz with latest opt_betagamma, using the same p as when get_approx_ratio_init first called
            solution_ansatz = ansatz_prog(opt_betagamma)
        else:
            # calculate solution_ansatz with latest opt_betagamma and its implied p
            p_new = len(opt_betagamma)//2
            ansatz_prog_flexible_p = ansatz_prog_init(prog_init, ListPauli_termsMy, n_qubits, n_destinations,   p_new,MyMixerHam)
            solution_ansatz = ansatz_prog_flexible_p(opt_betagamma)
            
        
        probsAbsolute  = WavefunctionSimulator().wavefunction(solution_ansatz).get_outcome_probs()
        opt_probs_absolute = [x for x in probsAbsolute.values()]
        show_debug=1
        for n in range(2**n_qubits):
        
            if opt_probs_absolute[n] > 0.00001 :    #|qn...q0>
                # calculation of energy, using n as the state, NOT recalculating the state by building its quantum circuit eg Program(X(0),I(1))
                Expectation_of_state = Energy_of_binary_state( n, SumPauli_termsMy,n_qubits)    

            if Expectation_of_state <= E_min_feas +  min_energy_constraint_problem:   
                
                #psi_opt, state_string = decimal_state_to_binary_reversed(n,n_qubits)
                prob_gnd_state += opt_probs_absolute[n]
            
            # If the Adjacency_constraint has an energy that is the minimium possible for a feasible state, the x mixer constraints have been met, ie space is 'feasible'..
            #...then the energy, without any problem_constraint energy, is averaged.
            if Energy_of_binary_state( n, SumPauli_terms_Xconstr, n_qubits) == energy_of_feasible_state: #use X mixer_c
                EV_feasible += opt_probs_absolute[n] * (Expectation_of_state - min_energy_constraint_problem )  #use problem's mixer_c
                prob_feasible_state +=opt_probs_absolute[n]
                n_feasible +=1
            elif show_debug:
                EV_not_feasible += opt_probs_absolute[n] * Expectation_of_state 
            if show_debug:EV_all_suggestions += opt_probs_absolute[n] * Expectation_of_state 
        if show_debug:
            approx_ratio_old = EV_feasible/(prob_feasible_state*E_min_feas)
            print('approx_ratio_old = ',approx_ratio_old,  '\nEV_feasible/prob_feasible_state = ', EV_feasible/prob_feasible_state, '\nEV_not_feasible =',EV_not_feasible,'\nEV_all_suggestions = ', EV_all_suggestions)

        approx_ratio = (EV_feasible/prob_feasible_state-E_max_feas) /(E_min_feas-E_max_feas)
        return prob_gnd_state, approx_ratio, prob_feasible_state
    return get_gnd_state_probs_and_approx_ratio


In [35]:

q5p1TradXmHqq = [6.898, -0.022] # nm -252.8 #11.1% Energy_max =-20 , Energy_min = -40 when constraint are complied with and have zero energy contribution to the cost function
q5p1RmDi = [3.9924273766343394, 2.827450565142573] # -38.4 EV of problem   71.1% r_approx new = 92.1%
q5p3RmDi = [3.963075998397324, 5.397047029891928, 2.3765397005374207, 3.1643560454228994, -0.03195512120218456, 0.30846008251700574] # -39.941 0.991 35, 0.997 = r_approx (E_max =0)
q5p3TradXmHqq =  [-2.113639856606643, -3.002768750687589, -4.828135143532405, 0.01965435129056827, 2.0763823207676677, 1.4405028404001305] #0.0370 4 570, = r_approx (E_max =0)
q11p1RmDi = [ 0.868, -0.02 ] #-1050.45 19.5%
q11p3RmDi   =[1.431421, 0.817087, 2.028664, 0.957856, 1.974961, 0.425763] # -1081.7 32.3=p_success, old approx=95.5, new approx = 93.3 
q11p3HW10_interp_TT_NM = [0.6286729798152555, 0.6094902517294243, 0.5605768275595322, -0.011444792772741205, -0.00425603944107247, -0.012365080443628575] #interp_TT -1067.7 37.0%
q11p3HW10_interp_NM = [0.6286897379372094, 0.6094509771046137, 0.5605711207713572, -0.011444854953217572, -0.0042559478813491616, -0.01236289525811517] #interp -1067.7 37.0%
q11p1TradXmH =  [2.4438652809143067, 1.5738661033630374]#-12623 1.0% CM140 new r_approx=43.8%
q11p3TradXmHqq =[-1.4926870960164018, -2.318859891143705, -5.881436314418812, 1.2582855789828138, 0.31574711088728313, 2.5490460457525597]  #-12293. 0.002 = gnd state prob,  0.955 = approx_ratio, CM140
q11p3TradXmH_interp_from_p1 =     [-0.6192225858511726, -0.3778830066802197, -7.7985735007499e-06, 1.573682853644343, 0.8154860817413727, -3.446929781736484e-05] #-12885.265 0.025271412390374358 1 300 
q11p3TradXmH_interp_TT_from_p1 =      [-0.37878142557194916, 8.906034487538886e-05, -0.6154068094481024, 1.57327276621201, 0.004972151376513632, -0.003760861752566373] #-13326.243 0.01261894177042601 0 312
q17p1TradH100angles = [-5.556807424465457, -0.0041934427604425565] #-377379.9,  0.007 = gnd state prob,  0.894 = approx_ratio,0.12758  =prob_feasible_state,  # 0.367= approx_ratio new 
q19p1RmDi = [0.982341, 3.147795]  #-9939.867=EV_min (12 repeats), 0.014, 3.0,  0.014 = gnd state prob,  0.969 = approx_ratio, 79.2 = approx_ratio new
opt_betagamma = q5p1RmDi#q11p3TradXmHqq
p = len(opt_betagamma)//2



# Get the Quantum circuit of the initial state (eg Hamamard for Xmixer)
prog_init = get_prog_init(n_qubits,remove_constraint,n_ambulance )
# Get Mixer quantum circuit
MyMixerHam, Ansatz_type = get_mixer(n_qubits,remove_constraint,n_ambulance )
# Get Pauli observables that represent an Ising problem, derived from an Adjacency table using a qubo definition of edge.
ListPauli_termsMy,SumPauli_termsMy  = Adjacency_qubo_to_Regetti( Adjacency, n_qubits)

if 1: print(SumPauli_termsMy)

ansatz_prog = ansatz_prog_init(prog_init, ListPauli_termsMy, n_qubits, n_destinations,   p, MyMixerHam) 

#Supply a 'Adjacency_feasible', whose energy in 'state_feasible' can be used to identify whether a given state is a feasible state.
# if state_feasible==None then all 2^n states of 'Adjacency_feasible' will be searched and the lowest energy will define a feasible state.
if n_qubits == 5 :state_feasible = '11011'
if n_qubits == 11 :state_feasible = '11111011111'
if n_qubits == 17 :state_feasible = '11111111011111111'
if n_qubits == 19 :state_feasible = '1111111110111111111'
if 1: state_feasible=None
# Initiate function to be used
if 1: get_gnd_state_probs_and_approx_ratio = get_approx_ratio_init(ansatz_prog,prog_init,ListPauli_termsMy,SumPauli_termsMy,n_qubits,p,Adjacency_constraint,Adjacency_feasible,  prt_details=0,state_feasible=state_feasible)
# Use function to calculate the ratios
if 1: 
    prob_gnd_state, approx_ratio, prob_feasible_state = get_gnd_state_probs_and_approx_ratio(opt_betagamma,prt_details=0)
    print('%4.4f'%prob_gnd_state, ' = prob_gnd_state,\n', '%4.4f'%approx_ratio, ' = approx_ratio,\n', '%4.4f'%prob_feasible_state, ' = prob_feasible_state,\n')

Dicke_state initial state 4  = HW
XY Ring mixer
(0.5+0j)*X0*X1 + (0.5+0j)*Y0*Y1 + (0.5+0j)*X1*X2 + (0.5+0j)*Y1*Y2 + (0.5+0j)*X2*X3 + (0.5+0j)*Y2*Y3 + (0.5+0j)*X3*X4 + (0.5+0j)*Y3*Y4 + (0.5+0j)*X0*X4 + (0.5+0j)*Y0*Y4
(-12.5+0j)*I + (-0.25+0j)*Z0*Z1 + (7.5+0j)*Z0 + (3.75+0j)*Z1 + (-1+0j)*Z0*Z2 + (2.5+0j)*Z2 + (-2.25+0j)*Z0*Z3 + (3.75+0j)*Z3 + (-4+0j)*Z0*Z4 + (7.5+0j)*Z4 + (-0.25+0j)*Z1*Z2 + (-1+0j)*Z1*Z3 + (-2.25+0j)*Z1*Z4 + (-0.25+0j)*Z2*Z3 + (-1+0j)*Z2*Z4 + (-0.25+0j)*Z3*Z4
approx_ratio_old =  0.9607943241686382 
EV_feasible/prob_feasible_state =  -38.43177296674553 
EV_not_feasible = -2.536765017992691e-30 
EV_all_suggestions =  -38.43177296674533
0.7115  = prob_gnd_state,
 0.9216  = approx_ratio,
 1.0000  = prob_feasible_state,



In [11]:
# Run the same problem as above but with a different p and angles
opt_betagamma = q11p3TradXmH_interp_from_p1
prob_gnd_state, approx_ratio, prob_feasible_state = get_gnd_state_probs_and_approx_ratio(opt_betagamma,prt_details=0)
print('%4.4f'%prob_gnd_state, ' = prob_gnd_state,\n', '%4.4f'%approx_ratio, ' = approx_ratio,\n', '%4.4f'%prob_feasible_state, ' = prob_feasible_state,\n')
print(get_gnd_state_probs_and_approx_ratio.__doc__)

0.9722340407199686  = approx_ratio_old -1069.4574447919654 -11025.274595671988 -12885.374231201218
0.0253  = prob_gnd_state,
 0.8889  = approx_ratio,
 0.1234  = prob_feasible_state,


        return A) the cumulative probability of the lowest energy state sampled, 
               B) 'approx_ratio' from the feasible states, (defined as states that minimize 'Adjacency_constraint') the average energy calculated without any constraint                       energy / ground state energy.
                       Or -1 if 'Adjacency_constraint' is not supplied
                       As defined by Wang in arXiv:1904.09314v2 [quant-ph] 21 May 2020, page 3 col 2
               C) The probability that one of the suggested states is a feasible state(ie complies with all constraint )
       type: float,float,float

       param
           ansatz_prog
               type:function that returns a Rigetti.Program()
           'Adjacency_constraint'  The adjacency table which contain only the constraint e